In [1]:
import pandas as pd
import re
from sklearn.metrics import f1_score, cohen_kappa_score
from tqdm import tqdm

In [2]:
#For arabic
files = ['PA_base'] + ['PA_'+str(i) for i in range(1,10)] 

def map_fn(row):
    row = str(row)
    if(re.search('نعم', row) and re.search('لا', row)):
        return None
    elif(re.search('نعم', row)):
        return 1
    elif(re.search('لا', row)):
        return 0

    return None

In [4]:
#For french
files = ['PF_base'] + ['PF_'+str(i) for i in range(1,10)] 

def map_fn(row):
    row = str(row)
    if(re.search('non', row) and re.search('haineuse', row)):
        return None
    elif(re.search('non', row) and re.search('contre', row)):
        return None
    elif(re.search('oui', row)):
        return 1
    elif(re.search('non', row)):
        return 0

    return None

In [6]:
#For german
files = ['PG_base'] + ['PG_'+str(i) for i in range(1,10)] 

def map_fn(row):
    row = str(row)
    if(re.search('ja', row)):
        return 1
    elif(re.search('nein', row)):
        return 0
        
    return None

In [8]:
#For hindi
files = ['PH_base'] + ['PH_'+str(i) for i in range(1,10)] 

def map_fn(row):
    row = str(row)
    if(re.search('हाँ', row) or re.search('हां', row) or re.search('जी', row)):
        return 1
    elif(re.search('नहीं', row) or re.search('गलत', row)):
        return 0

    return None

In [9]:
# First one cell of a language then run this cell and repeat it for all languages
for f in tqdm(files):
    df = pd.read_excel('../ChatGPT_SL/'+f+'.xlsx')
    col = df.columns[2:3]
    
    no_ans = []
    F1 = []
    IAA = []
    HS = []
    
    for i in col:
        df[i] = df[i].apply(lambda x: str(x).lower())
        df[i] = df[i].apply(map_fn)

        pen = (df.shape[0]-df[i].isna().sum())/df.shape[0]
        no_ans.append(df[i].isna().sum())

        tmp = df[['label', i]].dropna()

        F1.append(pen*f1_score(tmp['label'], tmp[i], average='weighted'))
        IAA.append(pen*cohen_kappa_score(tmp['label'], tmp[i]))
        HS.append(tmp[tmp[i]==1].shape[0]/tmp.shape[0])
    
    res = pd.read_csv('Performance_SL.csv')
    res[f+' Hal'] = no_ans
    res[f+' F1'] = F1
    res[f+' IAA'] = IAA
    res[f+' HS'] = HS
    res.to_csv('Performance_SL.csv', index=False)

100%|██████████| 10/10 [00:00<00:00, 10.83it/s]


In [13]:
# Sample code to write up the mapping function

for f in files:
    df = pd.read_excel('../ChatGPT_SL/'+f+'.xlsx')

    print(df.gpt.value_counts())
    print(df.gpt.value_counts().index)

gpt
हाँ,      192
हां,       82
घृ         61
जी ह       26
हाँ        26
हां        16
असत        13
हाँ।       12
विव         8
वास         8
नहीं        7
असह         7
शर्म        7
असहम        7
अव          4
गलत         2
उत्त        2
जाह         2
असली        1
जीतो        1
हां, ह      1
हाँ.        1
नकार        1
गैर         1
हिंस        1
ओले         1
हां।        1
हाँ, ह      1
जीत         1
ठीक         1
होगा        1
अत्य        1
वामप        1
जान         1
विच         1
असहन        1
Name: count, dtype: int64
Index(['हाँ,', 'हां,', 'घृ', 'जी ह', 'हाँ', 'हां', 'असत', 'हाँ।', 'विव', 'वास',
       'नहीं', 'असह', 'शर्म', 'असहम', 'अव', 'गलत', 'उत्त', 'जाह', 'असली',
       'जीतो', 'हां, ह', 'हाँ.', 'नकार', 'गैर', 'हिंस', 'ओले', 'हां।',
       'हाँ, ह', 'जीत', 'ठीक', 'होगा', 'अत्य', 'वामप', 'जान', 'विच', 'असहन'],
      dtype='object', name='gpt')
gpt
हाँ,      266
हां,       79
हाँ        46
हां        39
हाँ।       35
नहीं       11
घृ          8
जी ह        4
हाँ, ह      4